In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from tensorflow.python.framework import ops
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.model_selection import GridSearchCV

In [2]:
class RBFKernelSVM(BaseEstimator, ClassifierMixin) :
    """SVM with RBF kernel Classifier implemented with Tensorflow and compatible to SkLearn Estimator"""
    
    def __init__(self, gamma, C=1.0, verbose=False) :
        self.gamma = gamma
        self.C = C
        self.verbose = verbose

    def fit(self, X, y) : 
        X, y = check_X_y(X, y)
        
        self.xData_ = X
        self.yTarget_ = y
        
        # Step 1: Reinitialize the Tensorflow graph
        ops.reset_default_graph()
        self.sess_ = tf.Session()
        
        # Step 2: Create the Placeholders
        featuresSize = np.shape(X)[1]
        trainingDataSize = np.shape(X)[0]
        self.xInput_ = tf.placeholder(shape=[None, featuresSize], dtype=np.float32)
        self.yInput_ = tf.placeholder(shape=[None, 1], dtype=np.int32)
        self.prediction_grid_ = tf.placeholder(shape=[None, featuresSize], dtype=np.float32)
        self.xInputIndexes_ = tf.placeholder(shape=[None], dtype=np.int32)
        
        # Step 3: Initialize the model parameters
        b = tf.Variable(tf.random_normal(shape=[trainingDataSize, 1], dtype=np.float32))
        bEpoch = tf.gather(b, self.xInputIndexes_)
        
        # Step 4: Compute Training Kernel
        gammVal = tf.negative(tf.constant(self.gamma))
        dist = tf.reduce_sum(tf.square(self.xInput_), 1)
        dist = tf.reshape(dist, [-1, 1])
        sq_dists = tf.add(tf.subtract(dist, tf.multiply(2.0, tf.matmul(self.xInput_, tf.transpose(self.xInput_)))), tf.transpose(dist))
        my_kernel = tf.exp(tf.multiply(gammVal, tf.abs(sq_dists)))
        
        # Step 5: Compute the Loss
        first_term = tf.reduce_sum(bEpoch)
        b_vec_cross = tf.matmul(bEpoch, tf.transpose(bEpoch))
        y_target_cross = tf.matmul(tf.cast(self.yInput_, np.float32), tf.transpose(tf.cast(self.yInput_, np.float32)))
        second_term = tf.multiply(0.5, tf.reduce_sum(tf.multiply(tf.multiply(y_target_cross, my_kernel), b_vec_cross)))
        loss = tf.subtract(second_term, first_term)
        
        # Step 6: Compute Prediction Kernel
        rA = tf.reshape(tf.reduce_sum(tf.square(self.xInput_), 1), [-1, 1])
        rB = tf.reshape(tf.reduce_sum(tf.square(self.prediction_grid_), 1), [-1, 1])
        pred_sq_dist = tf.add(tf.subtract(rA, tf.multiply(2.0, tf.matmul(self.xInput_, tf.transpose(self.prediction_grid_)))), tf.transpose(rB))
        pred_kernel = tf.exp(tf.multiply(gammVal, tf.abs(pred_sq_dist)))
        
        # Step 7: Compute the Prediction
        prediction_output = tf.matmul(tf.multiply(tf.transpose(tf.cast(self.yInput_, np.float32)), tf.transpose(bEpoch)), tf.transpose(pred_kernel))
        self.prediction_ = tf.squeeze(tf.sign(prediction_output - tf.reduce_mean(prediction_output)))
        
        # Step 8: Optimizer
        optimizer = tf.train.GradientDescentOptimizer(0.001)
        train_step = optimizer.minimize(loss)
        
        # Step 9: Update the B variable
        tf.scatter_update(b, self.xInputIndexes_, bEpoch)
        
        # Step 10: Initialize the session
        init = tf.global_variables_initializer()
        self.sess_.run(init)
        
        # Step 10: Train the model
        epochs = 1000
        batchSize = 500
        np.random.seed(seed=2)
        for epoch in range(epochs) :
            stepCounter = 0
            indexes = np.arange(len(y))
            np.random.shuffle(indexes)
            currIndex = batchSize
            while currIndex <= len(y) :
                stepCounter = stepCounter + 1
                currIndexes = indexes[currIndex-batchSize:currIndex]
                batchX = X[currIndexes]
                batchY = np.reshape(y[currIndexes], newshape=[-1, 1])
                _, lossVal, predictionValue = self.sess_.run([train_step, loss, self.prediction_], feed_dict={self.xInput_:batchX, self.yInput_:batchY, self.prediction_grid_:batchX, self.xInputIndexes_:currIndexes})
                f1Score = f1_score(batchY, predictionValue, average='weighted')
                accuracy = accuracy_score(batchY, predictionValue)
                if self.verbose :
                    print("Epoch {0}, Step {1}    Loss: {2}    F1_Score: {3}   Accuracy: {4}".format(epoch, stepCounter, lossVal, f1Score, accuracy))
                currIndex = currIndex + batchSize
        
        if self.verbose :
            print("End of the training")
            
        return self
        
    def predict(self, X) :
        X = check_array(X)
        check_is_fitted(self, ['xData_'])
        output = self.sess_.run(self.prediction_, feed_dict={self.xInput_:X, self.yInput_:np.reshape(self.yTarget_, newshape=[-1, 1]), self.prediction_grid_:self.xData_, self.xInputIndexes_:np.arange(len(self.yTarget_))})  
        return output
    
    def score(self, X, y) :
        prediction = self.predict(X)
        accuracy = accuracy_score(y, prediction)
        if self.verbose :
            print("Accuracy Score is: {0}".format(accuracy))
        return f1_score(y, prediction, average='weighted')
    
    def classificationReport(self, X, y) :
        prediction = self.predict(X)
        return classification_report(y, prediction, output_dict=True)
        

In [3]:
# Build a Training and Test sets
dfTrain = pd.read_pickle('../data/dfTrain.pickle')
dfTest = pd.read_pickle('../data/dfTest.pickle')

trainData = dfTrain[['featureOne', 'featureTwo', 'featureThree', 'featureFour']].values
trainLabel = dfTrain['label'].values
testData = dfTest[['featureOne', 'featureTwo', 'featureThree', 'featureFour']].values
testLabel = dfTest['label'].values

print("The shape of the training data is {}".format(trainData.shape))
print("The shape of the training labels is {}".format(trainLabel.shape))
print("The shape of the test data is {}".format(testData.shape))
print("The shape of the test label is {}".format(testLabel.shape))

The shape of the training data is (3089, 4)
The shape of the training labels is (3089,)
The shape of the test data is (4000, 4)
The shape of the test label is (4000,)


In [7]:
# Perform coarse GridSearch to search for the optimal region of the parameters C and gamma
# Search C on [2^-3, ..., 2^3] and gamma on [2^-3, ..., 2^3]
cRange = np.arange(-3, 3, dtype=np.float32)
gRange = np.arange(-3, 3, dtype=np.float32)
paramsDict = {'gamma':np.power(2.0, gRange), 'C':np.power(2.0, cRange)}
svm = RBFKernelSVM(1.0)
grid = GridSearchCV(estimator=svm, param_grid=paramsDict, cv=5)
grid.fit(trainData, trainLabel)
print(grid.best_params_)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined an

{'C': 0.125, 'gamma': 0.5}


In [8]:
# Train SVM model with the Optimal Parameter value
model = RBFKernelSVM(gamma=0.5, C=0.125, verbose=True)
model.fit(trainData, trainLabel)

Epoch 0, Step 1    Loss: 263.3656921386719    F1_Score: 0.3558393853177091   Accuracy: 0.348
Epoch 0, Step 2    Loss: 265.7962341308594    F1_Score: 0.7805421809981145   Accuracy: 0.774
Epoch 0, Step 3    Loss: 195.8357696533203    F1_Score: 0.4990882869527385   Accuracy: 0.488
Epoch 0, Step 4    Loss: 202.00991821289062    F1_Score: 0.28009012960472374   Accuracy: 0.266
Epoch 0, Step 5    Loss: 264.8587646484375    F1_Score: 0.7435269039467735   Accuracy: 0.74
Epoch 0, Step 6    Loss: 214.30189514160156    F1_Score: 0.7056127780265713   Accuracy: 0.7
Epoch 1, Step 1    Loss: 159.046142578125    F1_Score: 0.6097208538587849   Accuracy: 0.6
Epoch 1, Step 2    Loss: 220.54205322265625    F1_Score: 0.5036289855072464   Accuracy: 0.496
Epoch 1, Step 3    Loss: 540.6625366210938    F1_Score: 0.6748187893081761   Accuracy: 0.668
Epoch 1, Step 4    Loss: 291.7872314453125    F1_Score: 0.6525493940874969   Accuracy: 0.646
Epoch 1, Step 5    Loss: 236.40211486816406    F1_Score: 0.5724113661782

Epoch 19, Step 4    Loss: 162.2606964111328    F1_Score: 0.7003501106589561   Accuracy: 0.698
Epoch 19, Step 5    Loss: 231.66261291503906    F1_Score: 0.7031064935064935   Accuracy: 0.696
Epoch 19, Step 6    Loss: 347.5869140625    F1_Score: 0.5806271159626643   Accuracy: 0.574
Epoch 20, Step 1    Loss: 226.1417999267578    F1_Score: 0.2783648266144619   Accuracy: 0.262
Epoch 20, Step 2    Loss: 217.57843017578125    F1_Score: 0.5783273797578433   Accuracy: 0.572
Epoch 20, Step 3    Loss: 131.51109313964844    F1_Score: 0.7893288365898067   Accuracy: 0.786
Epoch 20, Step 4    Loss: 289.7291259765625    F1_Score: 0.7217709349071936   Accuracy: 0.718
Epoch 20, Step 5    Loss: 208.92166137695312    F1_Score: 0.4362962106014547   Accuracy: 0.418
Epoch 20, Step 6    Loss: 133.95742797851562    F1_Score: 0.7166636736287332   Accuracy: 0.708
Epoch 21, Step 1    Loss: 175.7319793701172    F1_Score: 0.5388506652865657   Accuracy: 0.528
Epoch 21, Step 2    Loss: 316.80133056640625    F1_Score: 

Epoch 39, Step 1    Loss: 244.83958435058594    F1_Score: 0.6108912567508393   Accuracy: 0.604
Epoch 39, Step 2    Loss: 109.0093002319336    F1_Score: 0.4548319185059423   Accuracy: 0.44
Epoch 39, Step 3    Loss: 175.32354736328125    F1_Score: 0.7145911655568545   Accuracy: 0.712
Epoch 39, Step 4    Loss: 88.77536010742188    F1_Score: 0.5324887024432703   Accuracy: 0.526
Epoch 39, Step 5    Loss: 152.97836303710938    F1_Score: 0.5590627928772258   Accuracy: 0.544
Epoch 39, Step 6    Loss: 174.05523681640625    F1_Score: 0.3097981534386993   Accuracy: 0.294
Epoch 40, Step 1    Loss: 129.7795867919922    F1_Score: 0.60944   Accuracy: 0.604
Epoch 40, Step 2    Loss: 102.41027069091797    F1_Score: 0.7277746061167748   Accuracy: 0.718
Epoch 40, Step 3    Loss: 201.474853515625    F1_Score: 0.6969200872397298   Accuracy: 0.69
Epoch 40, Step 4    Loss: 243.8138885498047    F1_Score: 0.5047801312804171   Accuracy: 0.5
Epoch 40, Step 5    Loss: 358.2333679199219    F1_Score: 0.501983022044

Epoch 56, Step 6    Loss: 166.32115173339844    F1_Score: 0.21729325339443387   Accuracy: 0.194
Epoch 57, Step 1    Loss: 197.81365966796875    F1_Score: 0.6431334995625186   Accuracy: 0.634
Epoch 57, Step 2    Loss: 125.63134765625    F1_Score: 0.26989023969754866   Accuracy: 0.254
Epoch 57, Step 3    Loss: 151.10743713378906    F1_Score: 0.6829104534157096   Accuracy: 0.678
Epoch 57, Step 4    Loss: 136.5559844970703    F1_Score: 0.7568199999999999   Accuracy: 0.748
Epoch 57, Step 5    Loss: 115.84054565429688    F1_Score: 0.6190937996820348   Accuracy: 0.612
Epoch 57, Step 6    Loss: 190.4303741455078    F1_Score: 0.600696123109438   Accuracy: 0.596
Epoch 58, Step 1    Loss: 120.78984069824219    F1_Score: 0.4021609985834836   Accuracy: 0.394
Epoch 58, Step 2    Loss: 61.89426803588867    F1_Score: 0.5925010334849112   Accuracy: 0.58
Epoch 58, Step 3    Loss: 135.7355194091797    F1_Score: 0.5392029852791242   Accuracy: 0.53
Epoch 58, Step 4    Loss: 133.85354614257812    F1_Score: 

Epoch 75, Step 4    Loss: 191.9518280029297    F1_Score: 0.38343726198967565   Accuracy: 0.374
Epoch 75, Step 5    Loss: 106.88935852050781    F1_Score: 0.5558309027610658   Accuracy: 0.552
Epoch 75, Step 6    Loss: 116.25929260253906    F1_Score: 0.689161075069509   Accuracy: 0.678
Epoch 76, Step 1    Loss: 135.8516845703125    F1_Score: 0.6041665966669248   Accuracy: 0.594
Epoch 76, Step 2    Loss: 439.1400146484375    F1_Score: 0.4455913686177895   Accuracy: 0.438
Epoch 76, Step 3    Loss: 222.64767456054688    F1_Score: 0.47538319927665107   Accuracy: 0.462
Epoch 76, Step 4    Loss: 65.7005615234375    F1_Score: 0.7959120027356662   Accuracy: 0.792
Epoch 76, Step 5    Loss: 195.68667602539062    F1_Score: 0.7612674819257949   Accuracy: 0.754
Epoch 76, Step 6    Loss: 154.15773010253906    F1_Score: 0.4959810233819044   Accuracy: 0.488
Epoch 77, Step 1    Loss: 188.5792694091797    F1_Score: 0.3229366988759614   Accuracy: 0.314
Epoch 77, Step 2    Loss: 271.6259765625    F1_Score: 0

...

Epoch 936, Step 3    Loss: 2.202789306640625    F1_Score: 0.6090058272003226   Accuracy: 0.602
Epoch 936, Step 4    Loss: -11.448997497558594    F1_Score: 0.8181640679337862   Accuracy: 0.814
Epoch 936, Step 5    Loss: -0.5732574462890625    F1_Score: 0.5280648729301481   Accuracy: 0.516
Epoch 936, Step 6    Loss: -16.887493133544922    F1_Score: 0.41527911490683234   Accuracy: 0.404
Epoch 937, Step 1    Loss: -25.63772201538086    F1_Score: 0.795043098450002   Accuracy: 0.794
Epoch 937, Step 2    Loss: 3.6547622680664062    F1_Score: 0.5386985652849553   Accuracy: 0.534
Epoch 937, Step 3    Loss: -14.907615661621094    F1_Score: 0.58647639520463   Accuracy: 0.584
Epoch 937, Step 4    Loss: 5.007774353027344    F1_Score: 0.7847001261034049   Accuracy: 0.776
Epoch 937, Step 5    Loss: -1.9494819641113281    F1_Score: 0.6946929363319607   Accuracy: 0.686
Epoch 937, Step 6    Loss: 8.879619598388672    F1_Score: 0.4962036163522013   Accuracy: 0.484
Epoch 938, Step 1    Loss: -8.3812255859

Epoch 953, Step 1    Loss: 42.14723205566406    F1_Score: 0.49703086910439853   Accuracy: 0.492
Epoch 953, Step 2    Loss: -22.2117919921875    F1_Score: 0.7745475468145204   Accuracy: 0.77
Epoch 953, Step 3    Loss: 66.02751159667969    F1_Score: 0.5803486929101821   Accuracy: 0.57
Epoch 953, Step 4    Loss: -31.866058349609375    F1_Score: 0.8203031298203712   Accuracy: 0.816
Epoch 953, Step 5    Loss: -3.4414443969726562    F1_Score: 0.48412255450258324   Accuracy: 0.474
Epoch 953, Step 6    Loss: -14.522430419921875    F1_Score: 0.7707213365683673   Accuracy: 0.764
Epoch 954, Step 1    Loss: 2.7716331481933594    F1_Score: 0.6255767982828333   Accuracy: 0.62
Epoch 954, Step 2    Loss: -2.4150924682617188    F1_Score: 0.6035086868686869   Accuracy: 0.596
Epoch 954, Step 3    Loss: -6.81597900390625    F1_Score: 0.8390077746029635   Accuracy: 0.836
Epoch 954, Step 4    Loss: -23.795917510986328    F1_Score: 0.5727853231284226   Accuracy: 0.564
Epoch 954, Step 5    Loss: 29.1347274780

Epoch 970, Step 6    Loss: -4.4091033935546875    F1_Score: 0.6736148390616474   Accuracy: 0.668
Epoch 971, Step 1    Loss: -12.482608795166016    F1_Score: 0.4636996607359174   Accuracy: 0.458
Epoch 971, Step 2    Loss: 66.06138610839844    F1_Score: 0.7095166039905841   Accuracy: 0.702
Epoch 971, Step 3    Loss: 9.598304748535156    F1_Score: 0.5775220393123484   Accuracy: 0.57
Epoch 971, Step 4    Loss: -2.9703750610351562    F1_Score: 0.8790413044943064   Accuracy: 0.876
Epoch 971, Step 5    Loss: 17.63616180419922    F1_Score: 0.4420252976190476   Accuracy: 0.428
Epoch 971, Step 6    Loss: 6.462837219238281    F1_Score: 0.4202899097399973   Accuracy: 0.406
Epoch 972, Step 1    Loss: -5.783843994140625    F1_Score: 0.7403615085691918   Accuracy: 0.734
Epoch 972, Step 2    Loss: -11.275585174560547    F1_Score: 0.6701388350621225   Accuracy: 0.664
Epoch 972, Step 3    Loss: 89.41244506835938    F1_Score: 0.6562383922383922   Accuracy: 0.654
Epoch 972, Step 4    Loss: -28.83633041381

Epoch 991, Step 2    Loss: -29.77300262451172    F1_Score: 0.5871405956752347   Accuracy: 0.576
Epoch 991, Step 3    Loss: 16.202102661132812    F1_Score: 0.4789431258396775   Accuracy: 0.46
Epoch 991, Step 4    Loss: -32.90697479248047    F1_Score: 0.793288484185036   Accuracy: 0.788
Epoch 991, Step 5    Loss: -14.065738677978516    F1_Score: 0.7124542053663571   Accuracy: 0.708
Epoch 991, Step 6    Loss: 14.548416137695312    F1_Score: 0.7803103728483568   Accuracy: 0.778
Epoch 992, Step 1    Loss: -19.487213134765625    F1_Score: 0.40576471115931806   Accuracy: 0.394
Epoch 992, Step 2    Loss: 2.2658309936523438    F1_Score: 0.8334916550793308   Accuracy: 0.83
Epoch 992, Step 3    Loss: -24.71231460571289    F1_Score: 0.7504986464781731   Accuracy: 0.742
Epoch 992, Step 4    Loss: -14.866878509521484    F1_Score: 0.6508475311991319   Accuracy: 0.648
Epoch 992, Step 5    Loss: -8.050426483154297    F1_Score: 0.5801938568870529   Accuracy: 0.57
Epoch 992, Step 6    Loss: -16.514266967

RBFKernelSVM(C=0.125, gamma=0.5, verbose=True)

In [9]:
modelReportOnTest = model.classificationReport(testData, testLabel)
print(modelReportOnTest)
with open('../data/svmWithRBFKernel.pickle', 'wb') as fp :
    pickle.dump(modelReportOnTest, fp)
print('Report Saved')

{'-1': {'precision': 0.9632571996027806, 'recall': 0.97, 'f1-score': 0.966616841056303, 'support': 2000}, '1': {'precision': 0.9697885196374623, 'recall': 0.963, 'f1-score': 0.9663823381836427, 'support': 2000}, 'accuracy': 0.9665, 'macro avg': {'precision': 0.9665228596201214, 'recall': 0.9664999999999999, 'f1-score': 0.9664995896199728, 'support': 4000}, 'weighted avg': {'precision': 0.9665228596201214, 'recall': 0.9665, 'f1-score': 0.9664995896199727, 'support': 4000}}
Report Saved
